In [125]:
####### https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/skflow
import pandas as pd

people = pd.read_csv("people.csv",parse_dates=['date'])
#data = pd.read_csv("act_train.csv",parse_dates=['date'])
trainSub = pd.read_csv("act_train.csv",nrows=10000, parse_dates=['date'])
test = pd.read_csv("act_test.csv",parse_dates=['date'])
act = test['activity_id']

In [126]:
trainSub.head()

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,outcome
0,ppl_100,act2_1734928,2023-08-26,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 76,0
1,ppl_100,act2_2434093,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
2,ppl_100,act2_3404049,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
3,ppl_100,act2_3651215,2023-08-04,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
4,ppl_100,act2_4109017,2023-08-26,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0


In [127]:
from datetime import datetime

trainSub.iloc[0]['date']
#trainSub['date'] = datetime.strptime(trainSub['date'], '%Y-%m-%d')

mergedTrain = trainSub.merge(right=people,on="people_id")
#mergedTrain['duration'] = mergedTrain.apply(mergedTrain[d].max() - mergedTrain[d].min())

In [128]:
#data = data.fillna("nop")
mergedTrain = mergedTrain.fillna("nop")
#mergedTrain.info()

In [129]:
#mergedTrain["char_1_x"].unique()
from sklearn import preprocessing
LabelEncoder = preprocessing.LabelEncoder
#for i in mergedTrain:
#    if  :
#        print(mergedTrain[i])
        #mergedTrain[i] = le.fit_transform(mergedTrain[i])
mergedTrain = mergedTrain.apply(LabelEncoder().fit_transform)
for d in ['date_x', 'date_y']:
    mergedTrain['duration'] = mergedTrain[d].max() - mergedTrain[d].min()

In [130]:
mergedTrain = mergedTrain.drop(["people_id",'activity_id'],axis=1)

In [131]:
X,y = mergedTrain.drop(["outcome"], axis=1),mergedTrain["outcome"]
print(X.shape)
print(y.shape)

(10000, 53)
(10000,)


In [132]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,test_size=.4)

In [133]:
from sklearn import metrics
from sklearn.ensemble import GradientBoostingClassifier

start_time = time.time()
gbm = GradientBoostingClassifier(n_estimators=200) #oob_score=True, n_estimators=20
gbm.fit(X_train, y_train)

y_pred = gbm.predict(X_test)

print(metrics.accuracy_score(y_test, y_pred))
print("--- %s seconds ---" % (time.time() - start_time))

0.9755
--- 2.232597827911377 seconds ---


In [134]:
gbm.fit(X,y)
print(metrics.accuracy_score(y, gbm.predict(X)))

0.979


In [135]:
X["pred" ] = gbm.predict(X)

In [138]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import pandas

start_time = time.time()
X_new = SelectKBest(chi2, k=10).fit_transform(X, y)
X_train = X_new
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.011018991470336914 seconds ---


In [139]:
import tensorflow.contrib.learn as skflow
import tensorflow as tf
import time
import os

tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=2))

os.system("taskset -p 0xff %d" % os.getpid())
           
start_time = time.time()

classifier = skflow.DNNClassifier(hidden_units=[10, 10, 10], n_classes=2)
classifier.fit(X_train, y_train)
score = metrics.accuracy_score(y_test, classifier.predict(X_test))
print("Accuracy: %f" % score)

print("--- %s seconds ---" % (time.time() - start_time))

Instructions for updating:
Pass `tf.contrib.learn.infer_real_valued_columns_from_input(x)` or `tf.contrib.learn.infer_real_valued_columns_from_input_fn(input_fn)` as `feature_columns`, where `x` or `input_fn` is your argument to `fit`, `evaluate`, or `predict`.
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/array_ops.py:1811: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  result_shape.insert(dim, 1)


IndexError: single positional indexer is out-of-bounds

In [ ]:
#y_pred = classifier.predict(X_test)
#print(metrics.classification_report(y_test,y_pred))

In [91]:
#from sklearn import metrics
#rf = GradientBoostingClassifier(n_estimators=2000) #oob_score=True, n_estimators=20
#rfk = RandomForestClassifier(oob_score=True, n_estimators=20)

#rf.fit(X_train, y_train)
#y_pred = gbm.predict(X_test)
#print(metrics.accuracy_score(y_test, y_pred))

#rfk.fit(X_new_train, y_new_train)
#y_new_pred = rfk.predict(X_new_test)
#print(metrics.accuracy_score(y_new_test, y_new_pred))

In [92]:
test.head()
import skflow

,date_x,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,char_8_x,...,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38,duration
0,3,0,42,2,7,1,6,1,1,16,...,1,1,1,1,1,1,1,1,76,1175
1,3,4,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,76,1175
2,89,0,4,1,7,4,6,1,1,5,...,1,1,1,1,1,1,1,1,90,1175
3,133,0,13,2,7,4,6,1,1,14,...,1,1,1,1,1,1,1,1,90,1175
4,90,4,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,90,1175


In [93]:
test = test.fillna("nop")
test = test.merge(right=people,on="people_id")
test = test.drop(["people_id",'activity_id'],axis=1)
test = test.apply(LabelEncoder().fit_transform)

for d in ['date_x', 'date_y']:
    test['duration'] = test[d].max() - test[d].min()

KeyError: 'people_id'

In [94]:
test.head()

,date_x,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,char_8_x,...,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38,duration
0,3,0,42,2,7,1,6,1,1,16,...,1,1,1,1,1,1,1,1,76,1175
1,3,4,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,76,1175
2,89,0,4,1,7,4,6,1,1,5,...,1,1,1,1,1,1,1,1,90,1175
3,133,0,13,2,7,4,6,1,1,14,...,1,1,1,1,1,1,1,1,90,1175
4,90,4,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,90,1175


In [95]:
fuit = classifier.predict(test)

In [96]:
#test = test.apply(LabelEncoder().inverse_transform)
submission = pd.DataFrame(act)


submission['outcome'] = fuit

In [98]:
submission['outcome'].unique()

array([0])

In [89]:
submission.to_csv('submission.csv',index=False)